In [101]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
# #from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [102]:
# Reading the data
# rheas_tamsat_raw=pd.read_csv('data/rheas_tamsat/ken_n_25_tamsat_cultivar1_fert5_2015_2023_maize.csv')
rheas_tamsat_raw=pd.read_csv('data/chirps/ken_n_25_chirps__newmask_2022_2023_maize.csv')

In [103]:
# change dates to months or years
rheas_tamsat_raw['planting']=pd.to_datetime(rheas_tamsat_raw['planting'], format='ISO8601')
rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='ISO8601')
rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='ISO8601')
rheas_tamsat_raw['year'] = rheas_tamsat_raw['planting'].dt.year
rheas_tamsat_raw['year'] = rheas_tamsat_raw['planting'].dt.year
# rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='%Y/%m/%d')


# calculate yield
rheas_tamsat_raw['yield']=rheas_tamsat_raw['gwad']/1000
rheas_tamsat_raw


,id,gid,cname,season,ccode,ensemble,planting,harvest,wsgd,gwad,year,yield
0,5287,35,Nyandarua,2021 2,KEN.35_1,7,2021-10-01,2022-01-01,0.000,3637,2021,3.637
1,5610,37,Samburu,2021 2,KEN.37_1,10,2021-10-01,2021-12-20,0.000,2443,2021,2.443
2,2112,14,Kilifi,2021 2,KEN.14_1,32,2021-10-01,2022-01-27,0.548,968,2021,0.968
3,7227,47,West Pokot,2021 2,KEN.47_1,27,2021-10-01,2022-01-14,0.659,4221,2021,4.221
4,4808,32,Nandi,2021 2,KEN.32_1,8,2021-10-01,2022-01-23,0.799,2042,2021,2.042
...,...,...,...,...,...,...,...,...,...,...,...,...
7515,3971,26,Meru,2023 1,KEN.26_1,11,2023-03-30,2023-06-15,0.000,2626,2023,2.626
7516,3509,22,Machakos,2023 1,KEN.22_1,29,2023-03-30,2023-06-14,0.309,2244,2023,2.244
7517,1593,10,Kajiado,2023 1,KEN.10_1,33,2023-03-30,2023-07-25,0.420,2448,2023,2.448
7518,1083,7,Garissa,2023 1,KEN.7_1,3,2023-03-30,2023-06-20,0.000,2388,2023,2.388


In [104]:
# unimodal season
season = ['NONE', 'NONE', 'MAM', 'MAM','MAM', 'NONE', 'NONE', 'NONE', 'NONE', 'OND', 'OND', 'OND']
month_to_season=dict(zip(range(1,13), season))
rheas_tamsat_raw['season2']= rheas_tamsat_raw['planting'].dt.month.map(month_to_season)

# # Extract data only for MAM and OND
rheas_tamsat_MAM= rheas_tamsat_raw.loc[rheas_tamsat_raw['season2']=='MAM']
rheas_tamsat_OND=rheas_tamsat_raw.loc[rheas_tamsat_raw['season2']=='OND']
# extract data for 2023 only
rheas_tamsat_MAM_2022=rheas_tamsat_MAM.loc[rheas_tamsat_MAM['year']==2022]
rheas_tamsat_MAM_2023=rheas_tamsat_MAM.loc[rheas_tamsat_MAM['year']==2023]
rheas_tamsat_OND_2022=rheas_tamsat_OND.loc[rheas_tamsat_OND['year']==2022]

# aggregate 
rheas_tamsat_MAM_2022=rheas_tamsat_MAM_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_tamsat_MAM_2022=rheas_tamsat_MAM_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_tamsat_OND_2022=rheas_tamsat_OND_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})

In [105]:
# Read shapefiles and rename columns
country_shapefile = gpd.read_file("data/shp/gadm41_KEN_0.shp")
county_shapefile = gpd.read_file("data/shp/gadm41_KEN_1.shp")


# Rename kenya_shp 
county_shapefile=county_shapefile.rename(columns={'NAME1':'cname'})


In [106]:
# Merge data into shapefiles
MAM_2022 = pd.merge(county_shapefile, rheas_tamsat_MAM_2022, on="cname")
MAM_2023 = pd.merge(county_shapefile, rheas_tamsat_MAM_2023, on="cname")
OND_2022 = pd.merge(county_shapefile, rheas_tamsat_OND_2022, on="cname")
MAM_2023

,GID1,GID_0,COUNTRY,cname,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,...,season,ccode,ensemble,planting,harvest,wsgd,gwad,year,yield,season2
0,KEN.1_1,KEN,Kenya,Baringo,NA,NA,County,County,30,KE.BA,...,2023 1,KEN.1_1,25,2023-03-01,2023-06-10,0.000,3577,2023,3.577,MAM
1,KEN.1_1,KEN,Kenya,Baringo,NA,NA,County,County,30,KE.BA,...,2023 1,KEN.1_1,38,2023-03-02,2023-06-08,0.000,3765,2023,3.765,MAM
2,KEN.1_1,KEN,Kenya,Baringo,NA,NA,County,County,30,KE.BA,...,2023 1,KEN.1_1,37,2023-03-02,2023-06-11,0.000,3495,2023,3.495,MAM
3,KEN.1_1,KEN,Kenya,Baringo,NA,NA,County,County,30,KE.BA,...,2023 1,KEN.1_1,7,2023-03-04,2023-06-13,0.000,3526,2023,3.526,MAM
4,KEN.1_1,KEN,Kenya,Baringo,NA,NA,County,County,30,KE.BA,...,2023 1,KEN.1_1,29,2023-03-06,2023-06-13,0.000,3551,2023,3.551,MAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,KEN.47_1,KEN,Kenya,West Pokot,NA,NA,County,County,24,KE.WP,...,2023 1,KEN.47_1,4,2023-03-26,2023-07-07,0.741,3703,2023,3.703,MAM
1876,KEN.47_1,KEN,Kenya,West Pokot,NA,NA,County,County,24,KE.WP,...,2023 1,KEN.47_1,18,2023-03-27,2023-07-09,0.684,3455,2023,3.455,MAM
1877,KEN.47_1,KEN,Kenya,West Pokot,NA,NA,County,County,24,KE.WP,...,2023 1,KEN.47_1,8,2023-03-27,2023-07-09,0.677,3639,2023,3.639,MAM
1878,KEN.47_1,KEN,Kenya,West Pokot,NA,NA,County,County,24,KE.WP,...,2023 1,KEN.47_1,35,2023-03-30,2023-07-15,0.000,3909,2023,3.909,MAM


In [94]:
geo_source_MAM_2022 = GeoJSONDataSource(geojson=MAM_2022.to_json())
geo_source_MAM_2023 = GeoJSONDataSource(geojson=MAM_2023.to_json())
geo_source_OND_2022 = GeoJSONDataSource(geojson=OND_2022.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

TypeError: Object of type Timestamp is not JSON serializable

In [ ]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p1 = figure(title='Long Rains  Season MAM Yield (Mt/Ha) - 2022',
           outer_width=650, outer_height=600)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source_MAM_2022,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('Province: ', '@cname'), ('Yield', '@yield')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [ ]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p2 = figure(title='Long Rains Season MAM Yield (Mt/Ha) - 2023',
           outer_width=650, outer_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source_MAM_2023,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('Province: ', '@cname'), ('Yield', '@yield')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [ ]:
# Short Rains Season 2022
color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p3 = figure(title='Short Rains Season OND Yield(Mt/Ha) - 2022',
           outer_width=650, outer_height=600)

p3.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p3.patches('xs','ys', source=geo_source_OND_2022,
          fill_color={'field': 'yield', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('County', '@cname'), ('Yield', '@yield')])
p3.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p3.add_layout(color_bar, 'right')

In [ ]:
# Short Rains Season 2022
# color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p4 = figure(title='Short Rains Season OND Yield(Mt/Ha) - 2021',
#            outer_width=650, outer_height=600)

# p4.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p4.patches('xs','ys', source=geo_source_OND,
#           fill_color={'field': 'yield', 'transform': color_mapper1},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('County', '@cname'), ('Yield', '@yield')])
# p4.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p4.add_layout(color_bar, 'right')

In [ ]:

# p=gridplot([[p1, p2], [p3, p4]], width=650, height=600)
show(p1)

In [ ]:
# from bokeh.io import save
# # specify the output file name and location
# save(p, filename='yield_map.html', title='Yield Data Map')

In [ ]:
# from bokeh.io import save

# specify the output file name and location
# save(p, filename='yield_map.html', title='Yield Data Map')